In [1]:
import scvi
import scgen
import rpy2
import scib
import scib_mod
import anndata
import logging
import warnings
import scanorama
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sb
import scrublet as scr
import doubletdetection
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
from anndata import AnnData
from os import PathLike, fspath

Global seed set to 0
/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)
/Users/temitopeleke/miniforge3/envs/scRNA_seq_meta_analysis/lib/python3.10/site-packages/seaborn/rcmod.py:400: DeprecationWarning: distutils Version classes are deprecated. Use packa

In [2]:
def get_sys_dpi(width, height, diag):
    '''
    obtain dpi of system
    
    w: width in pixels (if unsure, go vist `whatismyscreenresolution.net`)
    h: height in pixels
    d: diagonal in inches
    '''
    w_inches = (diag**2/ (1 + height**2/width**2))**0.5
    return round(width/w_inches)

In [3]:
# # Ignore R warning messages
# #Note: this can be commented out to get more verbose R output
# rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# # Automatically convert rpy2 outputs to pandas dataframes
# pandas2ri.activate()
# anndata2ri.activate()
# %load_ext rpy2.ipython

warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

rcParams['figure.dpi'] = get_sys_dpi(1512, 982, 14.125)
rcParams['figure.figsize']=(5,5) #rescale figures

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

-----
anndata     0.8.0
scanpy      1.9.1
-----
PIL                         9.2.0
absl                        NA
adjustText                  NA
anndata2ri                  1.1
annoy                       NA
appnope                     0.1.2
asttokens                   NA
attr                        22.1.0
backcall                    0.2.0
beta_ufunc                  NA
binom_ufunc                 NA
cffi                        1.15.1
chex                        0.1.5
colorama                    0.4.5
cycler                      0.10.0
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.5.1
decorator                   5.1.1
defusedxml                  0.7.1
deprecate                   0.3.2
deprecated                  1.2.13
docrep                      0.3.2
doubletdetection            4.2
entrypoints                 0.4
etils                       0.8.0
executing                   0.8.3
fbpca                       NA
flax                      

## Table of contents:

  * <a href=#Reading>1. Reading in the data</a>
  * <a href=#Eval>2.2 Integration Evaluation</a> 
    * <a href=#SI >2.1 ....</a>


# 1. Reading in the data

### Read integrated data from `../data/processes/integrated` and `../data/processes/basic` directory

Processed data was obtained after performing basic `QC`, `filtering`, `normalization` and `high;y variable gene identification`. directory

Integrated data is obtained after integrating with eith `Scanorama`, `scVI`, or `scANVI`.

In [4]:
method = 'scanorama'
adata_integrated = sc.read_h5ad(f'../data/processed/integrated/adata_{method}_integrated.h5ad')

In [5]:
adata_query = {}
query_keys = ['grubman_etx', 'leng_etx', 'leng_sfg', 'mathys_pfc']

for key in query_keys:
    adata_query[key] = sc.read_h5ad(f'../data/processed/basic/adata_{key}_not_integrated.h5ad')

# 2. Evaluate Integration

To precisely evaluate the integration accuracy, we implement performance metrics from the `scIB` manuscrip using `scib.metrics`.

The following metrics are considered;


We implemented different metrics for evaluating batch correction and biological conservation in the `scib.metrics`
module.

<table class="docutils align-default">
  <colgroup>
    <col style="width: 50%" />
    <col style="width: 50%" />
  </colgroup>
  <thead>
    <tr class="row-odd"><th class="head"><p>Biological Conservation</p></th>
      <th class="head"><p>Batch Correction</p></th>
    </tr>
  </thead>
  <tbody>
    <tr class="row-even" >
      <td><ul class="simple">
        <li><p>Cell type ASW (<b><em>~6m 21s</em></b>)</p></li>
        <li><p>Cell cycle conservation (<b><em>~2m 15s</em></b>)</p></li>
        <li><p>Graph cLISI (<b><em>~2m 53s</em></b>)</p></li>
        <li><p>Adjusted rand index (ARI) for cell label (<b><em>~2m 25s</em></b>)</p></li>
        <li><p>Normalised mutual information (NMI) for cell label (<b><em>~2m 25s</em></b>)</p></li>
        <li><p>Highly variable gene conservation (<b><em>~1m 33s</em></b>)</p></li>
        <li><p>Isolated label ASW (<b><em>~30m 21s</em></b>)</p></li>
        <li><p>Isolated label F1 (<b><em>~5m 10s</em></b>)</p></li>
        <li><p>Trajectory conservation (<b><em>~15m 25s</em></b>)</p></li>
      </ul></td>
      <td><ul class="simple">
        <li><p>Batch ASW (<b><em>fast</em></b>)</p></li>
        <li><p>Principal component regression (<b><em>~1m 45.9s</em></b>)</p></li>
        <li><p>Graph iLISI (<b><em>~2m 56s</em></b>)</p></li>
        <li><p>Graph connectivity (<b><em>~51s</em></b>)</p></li>
        <li><p>kBET (K-nearest neighbour batch effect) (<b><em>~105m 10s</em></b>)</p></li>
      </ul></td>
    </tr>
  </tbody>
</table>



Thus, multiple metrics could be computed for each category of batch effect removal, label conservation and label-free conservation (Supplementary Table 2).  `Overall accuracy scores were computed by taking the weighted mean of all computed metrics, with a 40/60 weighting of batch effect removal to biological variance conservation (bio-conservation) irrespective of the number of metrics computed.`

In [6]:

def eval_performance(adata_int: AnnData,
                    adata_query: dict={}, 
                    batch_key: str='study',
                    label_key: str='cell_type',
                    kwargs: dict={}):

    split = [data for data in adata_query.values()]
    categories = list(adata_int.obs[batch_key].unique())
    adata_raw = scib.utils.merge_adata(*split, batch_key=batch_key, join="outer", batch_categories=categories, index_unique=None)

    del split
    del categories
    
    metrics = scib_mod.metrics.metrics(adata_raw, adata_int,  batch_key, label_key, **kwargs)

    return metrics

In [7]:
metrics_kwargs = {"embed": "X_scanorama", # embedding representation of adata_int 
                "cluster_key": "scib_cluster", # name of column to store cluster assignments. Will be overwritten if it exists
################ Bio conservation Metrics #############################
                "ari_": True, # whether to compute Ajusted Rand Index
                 "nmi_": True, # whether to compute Normalized Mutual Information (NMI)
                "nmi_method": "arithmetic", # which implementation of NMI to use
                "silhouette_": True, # whether to compute the average silhouette width scores for labels and batch
                "si_metric": "euclidean", # which distance metric to use for silhouette scores
                "cell_cycle_": True, # whether to compute cell cycle score conservation
                "compute_dpt": False, # whether to compute diffusion maps and pseudotime estimates for trajectory score estimates
                "trajectory_": False, # whether to compute trajectory score
                "organism": "human", # organism of the datasets, used for computing cell cycle scores on gene names
                "hvg_score_": True, # whether to compute highly variable gene conservation
                "isolated_labels_": False, # whether to compute both isolated label scores            
                "isolated_labels_f1_": True, # whether to compute isolated label score based on F1 score of clusters vs labels
                "isolated_labels_asw_": False, # whether to compute isolated label score based on ASW (average silhouette width)
                "lisi_graph_": False, # whether to compute both (graph integration local inverse Simpson’s Index) cLISI and iLISI
                "ilisi_": True, # whether to compute iLISI
                "get_max_f1_": True, # whether to compute F1 score on clustering
                "n_isolated": None, # maximum number of batches per label for label to be considered as isolated

################ Batch Correction Metrics #############################
                "pcr_": True, # whether to compute principal component regression using
                "graph_conn_": True, # whether to compute graph connectivity score
                "kBET_": False, # whether to compute (k-nearest-neighbor batch effect test) kBET score 
                "clisi_": True , # whether to compute cLISI
                "subsample": 0.5, # subsample fraction for LISI scores
                "n_cores": 1, # number of cores to be used for LISI functions
                "type_": 'embed', # one of 'full', 'embed' or 'knn' (used for kBET and LISI scores)
                "verbose": False}

In [8]:
metrics = eval_performance(adata_integrated, adata_query, batch_key='study', label_key='cell_type', kwargs=metrics_kwargs)

running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 28 clusters and added
    'scib_cluster', the cluster labels (adata.obs, categorical) (0:00:53)
running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 40 clusters and added
    'scib_cluster', the cluster labels (adata.obs, categorical) (0:00:50)
running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 47 clusters and added
    'scib_cluster', the cluster labels (adata.obs, categorical) (0:00:50)
running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 55 clusters and added
    'scib_cluster', the cluster labels (adata.obs, categorical) (0:00:47)
running Louvain clustering
    using the "louvain" package of Traag (2017)
    finished: found 61 clusters and added
    'scib_cluster', the cluster labels (adata.obs, categorical) (0:00:48)
NMI...
ARI...
Silhouette score...
PC regressi

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:35)
iLISI score...
computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:18)
filtered out 458 genes that are detected in less than 1 cells
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)
filtered out 1 genes that are detected in less than 1 cells
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)
filtered out 1 genes that are detected in less than 1 cells
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)
filtered out 206 genes that are detected in less than 1 cells
If you pass `n_top_genes`, all cutoffs are ignored.

In [15]:
metrics.reset_index(inplace=True)
metrics.rename(columns={"index": "method"}, inplace=True)
metrics

,level_0,method,0
0,0,NMI_cluster/label,0.510474
1,1,ARI_cluster/label,0.218149
2,2,ASW_label,0.543160
3,3,ASW_label/batch,0.820731
4,4,PCR_batch,1.000000
5,5,cell_cycle_conservation,0.592322
6,6,isolated_label_F1,0.156740
7,7,isolated_label_silhouette,NaN
8,8,graph_conn,0.925540
9,9,kBET,NaN
